In [4]:
import re
import unicodedata

import numpy as np
import pandas as pd
from rapidfuzz import fuzz, process

pd.set_option('display.max_colwidth', None)  # Show full content of each cell
pd.set_option('display.expand_frame_repr', False)  # Avoid line wrapping

# medicine list

In [1]:
medicine_list = [
    # English Medicines
    "Cyclosporine", "Mebendazole", "Pilocarpine", "Calcium", "Hydrocortisone", "Memantine",
    "Hydroquinone", "Loratadine", "Guaifenesin", "Retinol", "Hydroxyzine", "Carbocisteine",
    "Montelukast", "Dexamethasone", "Niacinamide", "Cefdinir", "Albendazole", "Gabapentin",
    "Levetiracetam", "Zinc", "Chlorhexidine", "Diclofenac", "Prednisolone", "Botox",
    "Dextromethorphan", "Lidocaine", "Metronidazole", "Acetylcysteine", "Ciprofloxacin",
    "Clindamycin", "Probiotics", "Erythromycin", "Paracetamol", "Omeprazole",
    "Fluorometholone", "Azithromycin", "Nystatin", "Valproate", "Pramipexole",
    "Carbamazepine", "Tretinoin", "Ofloxacin", "Tobramycin", "Timolol", "Ropinirole",
    "Brimonidine", "Pregabalin", "Rivastigmine", "Amantadine", "Ranitidine",
    "Salbutamol", "Domperidone", "Vitamin", "Dorzolamide", "Cetirizine", "Cefixime",
    "Topiramate", "Iron", "Latanoprost", "Multivitamins", "Donepezil", "Hyaluronic",
    "Ibuprofen", "Ondansetron", "Mupirocin", "Amoxicillin", "Doxycycline","Carbachol", 
    "Oral Amoxicillin",
    
    # Arabic Medicines
    "حمض الجليكوليك", "حمض الساليسيليك", "الببتيدات",
    "أملاح", "إريثرومايسين", "باراسيتامول", "دوكسيسيكلين", "ميبيندازول",
    "الزنك", "كاربوسيستين", "رانيتيدين", "أوندانسيترون", "مونتيلوكاست",
    "كلورهيكسيدين", "ديكلوفيناك", "ليدوكايين", "بريجابالين", "أزيثروميسين",
    "فيتامين", "نياسيناميد", "غوايفينيسين", "فالبروات", "أموكسيسيلين",
    "هيدروكسيزين", "البوتوكس", "لاتانوبروست", "توبراميسين", "جابابنتين",
    "ديكساميثازون", "ديكستروميثورفان", "سيبروفلوكساسين", "ليفيتيراسيتام",
    "الكالسيوم", "تيمولول", "روبينيرول", "الهيالورونيك", "كاربامازيبين",
    "نيستاتين", "إيبوبروفين", "بريمونيدين", "ألبيندازول", "هيدروكورتيزون",
    "ريفاستجمين", "موبيروسين", "كليندامايسين", "أسيتيل", "هيدروكينون",
    "البروبيوتيك", "ميترونيدازول", "دونيبيزيل", "أمانتادين", "بيلوكاربين",
    "سيكلوسبورين", "تريتينوين", "ريتينول", "الحديد", "أوفلوكساسين",
    "فلوروميثولون", "بريدنيزولون", "سيفيكسيم", "سيفدينير", "كربونات", "كارباشول", 
]

# frequency list

In [2]:
frequency_patterns = [
    ## 🔹 English Frequencies
    r"\bevery\s\d+\shours?\b",            # "every 6 hours"
    r"\bEvery\shours\b",
    r"\bevery\s\d+\s?-\s?\d+\shours?\b",  # "every 4-6 hours"
    r"\bonce\sdaily\b",                   # "once daily"
    r"\btwice\sdaily\b",                  # "twice daily"
    r"\bthree\stimes\sa\sday\b",          # "three times a day"
    r"\bfour\stimes\sa\sday\b",           # "four times a day"
    r"\b\d+\stimes\sa\sday\b",            # "5 times a day"
    
    ## ⏳ Time-Based
    r"\bevery\sother\sday\b",             # "every other day"
    r"\bevery\s\d+\sdays?\b",             # "every 3 days"
    r"\bevery\s\d+\sweeks?\b",            # "every 2 weeks"
    r"\bevery\s\d+\smonths?\b",           # "every 6 months"
    
    ## 🌙 Morning/Evening
    r"\bin\sthe\smorning\b",              # "in the morning"
    r"\bin\sthe\sevening\b",              # "in the evening"
    r"\bin\sthe\safternoon\b",            # "in the afternoon"
    r"\bin\sthe\snight\b",                # "in the night"
    r"\bdaily\sat\snoon\b",               # "daily at noon"
    
    ## 🍽 Meal-Based
    r"\bbefore\smeals?\b",                # "before meals"
    r"\bbefore\sbreakfast?\b",            # "before breakfast"
    r"\bafter\smeals?\b",                 # "after meals"
    r"\bafter\sbreakfast?\b",             # "after breakfast"
    r"\bbefore\sfood\b",                  # "before food"
    r"\bafter\sfood\b",                   # "after food"
    r"\bon\san\sempty\sstomach\b",        # "on an empty stomach"
    
    ## 🌙 Sleep
    r"\bbefore\sbedtime\b",               # "before bedtime"
    r"\bat\sbedtime\b",                   # "at bedtime"
    r"\bbefore\sgoing\sto\sbed\b",        # "before going to bed"
    
    ## 🔄 PRN (As Needed)
    r"\bas\sneeded\b",                    # "as needed"
    r"\bif\sneeded\b",                    # "if needed"
    r"\bwhen\snecessary\b",               # "when necessary"
    r"\bwhen\srequired\b",                # "when required"
    r"\bwhen\sfeeling\spain\b",           # "when feeling pain"
    
    ## 🚑 Perioperative
    r"\bbefore\ssurgery\b",               # "before surgery"
    r"\bafter\ssurgery\b",                # "after surgery"
    r"\bbefore\san\soperation\b",         # "before an operation"
    r"\bafter\san\soperation\b",          # "after an operation"

    ## 🔹 Arabic Frequencies
    r"\bكل\s\d+\sساعة\b",              # "كل 8 ساعات" (every X hours)
    r"\bكل\s\d+\s?-\s?\d+\sساعات?\b",  # "كل 4-6 ساعات" (every X-Y hours)
    r"\bمرة\sيوميا\b",                 # "مرة يوميًا" (once daily)
    r"\bمرة\sكل\sيوم\b",               # "مرة كل يوم" (once per day)
    r"\bمرة\sيوميا\b",                 # "مرة يوميا" (once per day)
    r"\bمرة\sأسبوعيا\b",               # "مرة أسبوعيًا" (once weekly)
    r"\bمرة\sكل\sأسبوع\b",             # "مرة كل أسبوع" (once per week)
    r"\bمرة\sشهريا\b",                 # "مرة شهريًا" (once monthly)
    r"\bمرة\sكل\sشهر\b",               # "مرة كل شهر" (once per month)
    r"\bمرتين\sيوميا\b",               # "مرتين يوميًا" (twice daily)
    r"\b\d+\sمرات?\sيوميا\b",          # "3 مرات يوميًا" (multiple times daily)
    
    ## ⏳ Time-Based
    r"\bكل\sيومين\b",                   # "كل يومين" (every other day)
    r"\bكل\s\d+\sأيام\b",               # "كل 3 أيام" (every X days)
    r"\bكل\s\d+\sأسابيع\b",             # "كل 2 أسابيع" (every X weeks)
    r"\bكل\s\d+\sشهور\b",               # "كل 6 شهور" (every X months)

   ## Arabic (Word-Based Numbers)
    r"\bكل\sساعة\b",                     # "كل واحدة ساعة" (every one hour)
    r"\bكل\sساعتين\b",                   # "كل اثنتين ساعة" (every two hours)
    r"\bكل\sثلاث\sساعات\b",               # "كل ثلاث ساعات" (every three hours)
    r"\bكل\sأربع\sساعات\b",              # "كل أربع ساعات" (every four hours)
    r"\bكل\sخمس\sساعات\b",               # "كل خمس ساعات" (every five hours)
    r"\bكل\sست\sساعات\b",                # "كل ست ساعات" (every six hours)
    r"\bكل\sسبع\sساعات\b",               # "كل سبع ساعات" (every seven hours)
    r"\bكل\sثماني\sساعات\b",             # "كل ثماني ساعات" (every eight hours)
    r"\bكل\sتسع\sساعات\b",               # "كل تسع ساعات" (every nine hours)
    r"\bكل\sعشر\sساعات\b",               # "كل عشر ساعات" (every ten hours)
    r"\bكل\sإحدى\sعشرة\sساعة\b",         # "كل إحدى عشرة ساعة" (every 11 hours)
    r"\bكل\sاثنتي\sعشرة\sساعة\b",        # "كل اثنتي عشرة ساعة" (every 12 hours)
    
    ## 🌙 Morning/Evening
    r"\bفي\sالصباح\b",                  # "في الصباح" (in the morning)
    r"\bفي\sالمساء\b",                  # "في المساء" (in the evening)
    r"\bفي\sالظهيرة\b",                 # "في الظهيرة" (at noon)
    r"\bفي\sالليل\b",                   # "في الليل" (at night)
    
    ## 🍽 Meal-Based
    r"\bقبل\sالأكل\b",                  # "قبل الأكل" (before meals)
    r"\bبعد\sالأكل\b",                  # "بعد الأكل" (after meals)
    r"\bقبل\sالطعام\b",                 # "قبل الطعام" (before food)
    r"\bبعد\sالطعام\b",                 # "بعد الطعام" (after food)
    r"\bعلى\sمعدة\sفارغة\b",            # "على معدة فارغة" (on an empty stomach)
    r"\bعلى\sالريق\b",                   # "على الريق" (fasting)
    
    ## 🌙 Sleep
    r"\bقبل\sالنوم\b",                 # "قبل النوم" (before sleep)
    r"\bعند\sالنوم\b",                 # "عند النوم" (at bedtime)
    
    ## 🔄 PRN (As Needed)
    r"\bعند\sاللزوم\b",                # "عند اللزوم" (as needed)
    r"\bحسب\sالحاجة\b",                # "حسب الحاجة" (as required)
    r"\bإذا\sاستدعت\sالحاجة\b",         # "إذا استدعت الحاجة" (if necessary)
    r"\bعند\sالشعور\sبالألم\b",          # "عند الشعور بالألم" (when feeling pain)
    
    ## 🚑 Perioperative
    r"\bقبل\sالعملية\b",              # "قبل العملية" (before surgery)
    r"\bبعد\sالعملية\b",              # "بعد العملية" (after surgery)
    r"\bقبل\sالتدخل\sالجراحي\b",     # "قبل التدخل الجراحي" (before an operation)
    r"\bبعد\sالتدخل\sالجراحي\b",     # "بعد التدخل الجراحي" (after an operation)
]

In [5]:
# Load dataset (Assuming CSV)
df = pd.read_excel("Train.xlsx")

# Specify the column containing extracted prescription text
target_column = "Prescription"

In [6]:
df[target_column].head

<bound method NDFrame.head of 0                                                                                Carbachol Every 8 hours
1         Ofloxacin كل ست  ساعات , Brimonidine كل ٦ ساعات , كارباشول عند اللزوم , Latanoprost مرة يومياً
2                                                                           Brimonidine Before breakfast
3          Carbachol عند اللزوم , Tobramycin عند اللزوم , Pilocarpine كل ٦ ساعات , لاتانوبروست قبل النوم
4                                                      Fluorometholone As needed , Latanoprost As needed
                                                     ...                                                
618    Doxycycline مرة يومياً , كليندامايسين قبل النوم , موبيروسين مرتين يومياً , كلورهيكسيدين قبل النوم
619                                                                              ديكلوفيناك مرتين يومياً
620                                                                                  Nystatin بعد الغداء
621                      

In [7]:
def extract_medicine_and_frequency(text):
    """
    Extracts medicine names and dosage frequency while maintaining the original order.
    Returns a list of dictionaries [{medicine: "name", frequency: "value"}].
    """
    if not isinstance(text, str):  # Ensure text is a string
        return []
    
    structured_output = []
    
    # Combine medicine names and frequency patterns into a single regex pattern
    combined_pattern = r"|".join(
        [re.escape(med) for med in medicine_list] + frequency_patterns
    )

    # Find all matches in the order they appear in the text
    matches = re.findall(combined_pattern, text, re.IGNORECASE)
    
    current_medicine = None  # Track last detected medicine
    
    for match in matches:
        if match in medicine_list:  
            current_medicine = match  # Update the latest detected medicine
            structured_output.append({"medicine": match, "frequency": ""})  # Default frequency
        
        elif current_medicine:  
            # Assign the frequency to the last detected medicine **only if it does not have one yet**
            if structured_output[-1]["frequency"] == "":
                structured_output[-1]["frequency"] = match  
            else:
                # If previous medicine already has a frequency, store this frequency separately
                structured_output.append({"medicine": current_medicine, "frequency": match})
    
    return structured_output

In [8]:
# Apply extraction function to dataset
df["structured_prescriptions"] = df[target_column].apply(extract_medicine_and_frequency)

# Convert the entire dataset into a list of lists of dictionaries
structured_output = df["structured_prescriptions"].tolist()

# Print the final structured output
df["structured_prescriptions"].head()

0                                                                                                                                                             [{'medicine': 'Carbachol', 'frequency': 'Every 8 hours'}]
1                [{'medicine': 'Ofloxacin', 'frequency': ''}, {'medicine': 'Brimonidine', 'frequency': ''}, {'medicine': 'كارباشول', 'frequency': 'عند اللزوم'}, {'medicine': 'Latanoprost', 'frequency': 'مرة يوميا'}]
2                                                                                                                                                        [{'medicine': 'Brimonidine', 'frequency': 'Before breakfast'}]
3    [{'medicine': 'Carbachol', 'frequency': 'عند اللزوم'}, {'medicine': 'Tobramycin', 'frequency': 'عند اللزوم'}, {'medicine': 'Pilocarpine', 'frequency': ''}, {'medicine': 'لاتانوبروست', 'frequency': 'قبل النوم'}]
4                                                                                                    [{'medicine': 'Fluorometholone', 'f

In [ ]:
def extract_medicine_and_frequency(text):
    if not isinstance(text, str):  # Ensure text is a string
        return []
    
    structured_output = []
    
    # Combine medicine names and frequency patterns into a single regex pattern
    combined_pattern = r"|".join(
        [re.escape(med) for med in medicine_list] + frequency_patterns
    )

    # Find all matches in the order they appear in the text
    matches = re.findall(combined_pattern, text, re.IGNORECASE)
    
    current_medicine = None  # Track last detected medicine
    
    for match in matches:
        if match in medicine_list:  
            current_medicine = match  # Update the latest detected medicine
            structured_output.append({"medicine": match, "frequency": ""})  # Default frequency
        
        elif current_medicine:
            # Assign the frequency to the last detected medicine **only if it does not have one yet**
            if structured_output[-1]["frequency"] == "":
                structured_output[-1]["frequency"] = match  
            else:
                # If previous medicine already has a frequency, store this frequency separately
                structured_output.append({"medicine": current_medicine, "frequency": match})
    
    return structured_output

def remove_diacritics(text):
    # Normalize the text to decompose characters into base characters and diacritics
    normalized_text = unicodedata.normalize('NFKD', text)
    
    # Filter out combining characters (diacritics)
    cleaned_text = ''.join(
        char for char in normalized_text
        if not unicodedata.combining(char)
    )
    return cleaned_text

In [15]:
text = "Ofloxacin كل ست ساعات , Brimonidine كل ٦ ساعات , كارباشول عند اللزوم , Latanoprost مرة يومياً"
# text = "Carbachol عند اللزوم , Tobramycin عند اللزوم , Pilocarpine كل ٦ ساعات , لاتانوبروست قبل النوم"
# text = "Doxycycline مرة يومياً , كليندامايسين قبل النوم , موبيروسين مرتين يومياً , كلورهيكسيدين قبل النوم"

text = remove_diacritics(text)
result = extract_medicine_and_frequency(text)
result

[{'medicine': 'Ofloxacin', 'frequency': 'كل ست ساعات'},
 {'medicine': 'Brimonidine', 'frequency': ''},
 {'medicine': 'كارباشول', 'frequency': 'عند اللزوم'},
 {'medicine': 'Latanoprost', 'frequency': 'مرة يوميا'}]

# extract function

In [16]:
DEFAULT_FREQUENCY = "Every 6 hours"

def extract_medicine_and_frequency(sentence, medicine_list, frequency_patterns):
    extracted_medicines = []
    extracted_frequencies = []
    medicine_positions = {}
    frequency_positions = {}

    # Step 1: Extract medicines and their positions
    for med in medicine_list:
        match = re.search(rf"\b{re.escape(med)}\b", sentence)
        if match:
            extracted_medicines.append(med)
            medicine_positions[med] = match.start()

    # Step 2: Extract frequencies and their positions
    for pattern in frequency_patterns:
        pattern_re = re.compile(pattern)
        matches = pattern_re.finditer(sentence)
        for match in matches:
            extracted_frequencies.append(match.group())
            frequency_positions[match.group()] = match.start()

    # Step 3: Sort medicines and frequencies by their position
    sorted_medicines = sorted(medicine_positions.items(), key=lambda x: x[1])
    sorted_frequencies = sorted(frequency_positions.items(), key=lambda x: x[1])

    # Step 4: Pair medicines with frequencies
    extracted_results = []
    freq_index = 0
    last_known_frequency = None
    
    for i, (med, med_pos) in enumerate(sorted_medicines):
        extracted_frequency = "Unknown"
        next_med_pos = sorted_medicines[i + 1][1] if i + 1 < len(sorted_medicines) else len(sentence)

        # Check if there are words between this medicine and the next
        words_between = sentence[med_pos + len(med):next_med_pos].strip()
        if words_between and len(words_between.split()) <= 3:
            extracted_frequency = words_between
        else:
            # Try to match frequency after the medicine (if ordered correctly)
            while freq_index < len(sorted_frequencies):
                freq, freq_pos = sorted_frequencies[freq_index]
                if freq_pos > med_pos and freq_pos < next_med_pos:
                    extracted_frequency = freq
                    last_known_frequency = freq
                    freq_index += 1
                    break
                freq_index += 1

            # If no direct match, use last known frequency
            if extracted_frequency == "Unknown" and last_known_frequency:
                extracted_frequency = last_known_frequency

            # If still unknown, use default
            if extracted_frequency == "Unknown":
                extracted_frequency = DEFAULT_FREQUENCY

        extracted_results.append((med, extracted_frequency))
    
    return extracted_results

In [ ]:
# text = "Doxycycline مرة يومياً , كليندامايسين قبل النوم , موبيروسين مرتين يومياً , كلورهيكسيدين قبل النوم"
text = "Paracetamol hours Every Oral Amoxicillin other lunch After Erythromycin Nystatin"

print(text)
text = remove_diacritics(text)
result = extract_medicine_and_frequency(text, medicine_list, frequency_patterns)
result

Paracetamol hours Every Oral Amoxicillin other lunch After Erythromycin Nystatin


[('Paracetamol', 'hours Every'),
 ('Oral Amoxicillin', 'Every 6 hours'),
 ('Amoxicillin', 'other lunch After'),
 ('Erythromycin', 'Every 6 hours'),
 ('Nystatin', 'Every 6 hours')]

# try with bert

In [26]:
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from datasets import Dataset
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
import numpy as np

class MedicalPrescriptionNER:
    def __init__(self, model_name="distilbert-base-multilingual-cased"):
        """
        Initialize the NER system for medical prescriptions
        """
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.ner_pipeline = None
        
        # For custom training
        self.labels = ["O", "B-MEDICATION", "I-MEDICATION", "B-FREQUENCY", "I-FREQUENCY"]
        self.id2label = {i: label for i, label in enumerate(self.labels)}
        self.label2id = {label: i for i, label in enumerate(self.labels)}
        
        # Arabic frequency phrases and their standardized mappings
        self.arabic_frequencies = {
            "كل ست ساعات": "every 6 hours",
            "كل ٦ ساعات": "every 6 hours",
            "عند اللزوم": "as needed",
            "مرة يومياً": "once daily",
            "قبل النوم": "before sleep",
            "قبل الإفطار": "before breakfast",
            "بعد الغداء": "after lunch",
            "كل ٨ ساعات": "every 8 hours"
        }
        
        # English medication names and their Arabic transliterations
        self.medication_mappings = {
            "كارباشول": "Carbachol",
            "أوفلوكساسين": "Ofloxacin",
            "لاتانوبروست": "Latanoprost",
            "بريمونيدين": "Brimonidine",
            "توبراميسين": "Tobramycin",
            "بيلوكاربين": "Pilocarpine",
            "فلوروميثولون": "Fluorometholone",
            "دورزولاميد": "Dorzolamide",
            "تيمولول": "Timolol",
            "بريدنيزولون": "Prednisolone"
        }
        
        # Initialize pattern matchers
        self._initialize_patterns()
    
    def _initialize_patterns(self):
        """Initialize regex patterns for medications and frequencies"""
        # Common medication patterns
        common_medications = [
            "Carbachol", "Ofloxacin", "Brimonidine", "Latanoprost", "Tobramycin",
            "Pilocarpine", "Fluorometholone", "Dorzolamide", "Timolol", "Prednisolone"
        ]
        
        # Build pattern for medication names (case insensitive)
        self.medication_pattern = re.compile(
            r'\b(' + '|'.join(common_medications) + r')\b', 
            re.IGNORECASE
        )
        
        # Common frequency patterns in English
        self.eng_frequency_pattern = re.compile(
            r'\b(every\s+\d+\s+hours?|'
            r'once daily|twice daily|'
            r'before (breakfast|lunch|dinner|sleep|bed)|'
            r'after (breakfast|lunch|dinner)|'
            r'as needed|daily|weekly|monthly|'
            r'\d+ times? (daily|a day))\b',
            re.IGNORECASE
        )
        
        # Arabic frequency patterns (needs simplified regex for demo)
        self.arabic_frequency_pattern = re.compile(
            '|'.join(map(re.escape, self.arabic_frequencies.keys()))
        )
    
    def load_pretrained_model(self):
        """Load a pretrained multilingual NER model"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(
            self.model_name, 
            num_labels=len(self.labels),
            id2label=self.id2label,
            label2id=self.label2id
        )
        self.ner_pipeline = pipeline(
            "ner", 
            model=self.model, 
            tokenizer=self.tokenizer,
            aggregation_strategy="simple"  # Merge subwords
        )
        
        print(f"Loaded pretrained model: {self.model_name}")
        
    def prepare_training_data(self, prescriptions):
        """
        Prepare training data from prescription list using pattern matching
        Returns data in format suitable for spaCy training
        """
        training_data = []
        
        for text in prescriptions:
            doc = {"text": text, "entities": []}
            
            # Find medications using regex
            for med_match in self.medication_pattern.finditer(text):
                doc["entities"].append({
                    "start": med_match.start(),
                    "end": med_match.end(),
                    "label": "MEDICATION"
                })
            
            # Find Arabic medication names
            for ar_med, en_med in self.medication_mappings.items():
                for match in re.finditer(re.escape(ar_med), text):
                    doc["entities"].append({
                        "start": match.start(),
                        "end": match.end(),
                        "label": "MEDICATION"
                    })
            
            # Find English frequencies
            for freq_match in self.eng_frequency_pattern.finditer(text):
                doc["entities"].append({
                    "start": freq_match.start(),
                    "end": freq_match.end(),
                    "label": "FREQUENCY"
                })
            
            # Find Arabic frequencies
            for ar_freq_match in self.arabic_frequency_pattern.finditer(text):
                doc["entities"].append({
                    "start": ar_freq_match.start(),
                    "end": ar_freq_match.end(),
                    "label": "FREQUENCY"
                })
            
            training_data.append(doc)
        
        return training_data
    
    def prepare_spacy_data(self, training_data):
        """Convert training data to spaCy format"""
        nlp = spacy.blank("xx")  # multilingual blank model
        db = DocBin()
        
        for example in training_data:
            text = example["text"]
            doc = nlp.make_doc(text)
            
            # Sort entities by start position
            entities = sorted(example["entities"], key=lambda e: e["start"])
            
            # Create span objects
            spans = [doc.char_span(e["start"], e["end"], label=e["label"]) 
                     for e in entities]
            spans = [span for span in spans if span is not None]
            
            # Filter overlapping spans
            filtered_spans = filter_spans(spans)
            
            # Add spans to document
            doc.spans["entities"] = filtered_spans
            db.add(doc)
        
        return db
    
    def train_spacy_model(self, training_data, output_dir="./med_ner_model"):
        """Train a spaCy model with the prepared data"""
        db = self.prepare_spacy_data(training_data)
        
        # Save converted training data
        db.to_disk("./train_data.spacy")
        
        # Command to run in shell (not executed here):
        print("Run the following command to train the model:")
        print(f"python -m spacy train config.cfg --output {output_dir} --paths.train train_data.spacy --paths.dev train_data.spacy")
    
    def train_transformer_model(self, training_data, output_dir="./med_ner_transformer"):
        """Train a transformer model with the prepared data"""
        # Convert to IOB format for transformer training
        tokenized_inputs = []
        tags = []
        
        for example in training_data:
            text = example["text"]
            tokens = []
            labels = ["O"] * len(text.split())  # Initialize all as Outside
            
            # Process each character position
            for i, char in enumerate(text):
                # Check if this position starts an entity
                for entity in example["entities"]:
                    if i == entity["start"]:
                        entity_tokens = text[entity["start"]:entity["end"]].split()
                        entity_idx = len(tokens)
                        for j, token in enumerate(entity_tokens):
                            if j == 0:
                                labels[entity_idx + j] = f"B-{entity['label']}"
                            else:
                                labels[entity_idx + j] = f"I-{entity['label']}"
                
                # Add token if it's a word boundary
                if char.isspace() or i == 0:
                    if i > 0 and text[i-1].isspace():
                        continue
                    word = ""
                    j = i
                    while j < len(text) and not text[j].isspace():
                        word += text[j]
                        j += 1
                    if word:
                        tokens.append(word)
            
            tokenized_inputs.append(tokens)
            tags.append(labels[:len(tokens)])  # Ensure labels match tokens
        
        # Convert to HuggingFace dataset format
        dataset_dict = {
            "tokens": tokenized_inputs,
            "ner_tags": tags
        }
        dataset = Dataset.from_dict(dataset_dict)
        
        # Print training info
        print(f"Created training dataset with {len(dataset)} examples")
        print("Sample tokens:", dataset[0]["tokens"])
        print("Sample tags:", dataset[0]["ner_tags"])
        
        # Code to train the transformer model (not executed here)
        print("\nTo train the transformer model, use HuggingFace Trainer...")
    
    def extract_entities_with_regex(self, text):
        """Extract medications and frequencies using regex patterns"""
        entities = []
        
        # Extract English medications
        for match in self.medication_pattern.finditer(text):
            entities.append({
                "entity": "MEDICATION",
                "value": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        
        # Extract Arabic medications
        for ar_med, en_med in self.medication_mappings.items():
            for match in re.finditer(re.escape(ar_med), text):
                entities.append({
                    "entity": "MEDICATION",
                    "value": ar_med,
                    "normalized": en_med,
                    "start": match.start(),
                    "end": match.end()
                })
        
        # Extract English frequencies
        for match in self.eng_frequency_pattern.finditer(text):
            entities.append({
                "entity": "FREQUENCY",
                "value": match.group(),
                "start": match.start(),
                "end": match.end()
            })
        
        # Extract Arabic frequencies
        for match in self.arabic_frequency_pattern.finditer(text):
            ar_freq = match.group()
            entities.append({
                "entity": "FREQUENCY",
                "value": ar_freq,
                "normalized": self.arabic_frequencies.get(ar_freq, ar_freq),
                "start": match.start(),
                "end": match.end()
            })
        
        # Sort by position in text
        entities.sort(key=lambda x: x["start"])
        
        return entities
    
    def pair_medications_with_frequencies(self, entities, text):
        """Pair medications with their frequencies based on proximity"""
        medications = [e for e in entities if e["entity"] == "MEDICATION"]
        frequencies = [e for e in entities if e["entity"] == "FREQUENCY"]
        
        pairs = []
        
        for med in medications:
            # Find closest frequency
            closest_freq = None
            min_distance = float('inf')
            
            for freq in frequencies:
                # Check if frequency follows medication
                if freq["start"] > med["end"]:
                    # Distance to next frequency
                    distance = freq["start"] - med["end"]
                    
                    # Check if there's another medication between this med and freq
                    has_med_between = any(
                        m["start"] > med["end"] and m["end"] < freq["start"] 
                        for m in medications if m != med
                    )
                    
                    if not has_med_between and distance < min_distance:
                        min_distance = distance
                        closest_freq = freq
                        
                        # If they're adjacent with only comma/whitespace, this is a strong match
                        sep_text = text[med["end"]:freq["start"]]
                        if re.match(r'^[\s,]*$', sep_text):
                            break
            
            # If no frequency found after, look for one before (less common but possible)
            if closest_freq is None:
                for freq in frequencies:
                    if freq["end"] < med["start"]:
                        distance = med["start"] - freq["end"]
                        
                        has_med_between = any(
                            m["end"] > freq["end"] and m["start"] < med["start"] 
                            for m in medications if m != med
                        )
                        
                        if not has_med_between and distance < min_distance:
                            min_distance = distance
                            closest_freq = freq
            
            # Only pair if the frequency is reasonably close
            if closest_freq and min_distance < 50:  # Arbitrary threshold
                pairs.append({
                    "medication": med.get("normalized", med["value"]),
                    "frequency": closest_freq.get("normalized", closest_freq["value"]),
                    "confidence": 1.0 - (min_distance / 100)  # Higher confidence for closer pairs
                })
            else:
                # Medication without clear frequency
                pairs.append({
                    "medication": med.get("normalized", med["value"]),
                    "frequency": "unspecified",
                    "confidence": 0.5
                })
        
        return pairs
    
    def process_prescription(self, text):
        """Process a single prescription text"""
        # Extract entities using regex (faster for prototype)
        entities = self.extract_entities_with_regex(text)
        
        # Pair medications with frequencies
        medication_frequency_pairs = self.pair_medications_with_frequencies(entities, text)
        
        return {
            "text": text,
            "entities": entities,
            "medication_schedule": medication_frequency_pairs
        }
    
    def batch_process(self, prescription_list):
        """Process a batch of prescriptions"""
        results = []
        for text in prescription_list:
            results.append(self.process_prescription(text))
        return results

# Example usage
def main():
    # Example prescription data
    prescriptions = [
        "Carbachol Every 8 hours",
        "Ofloxacin كل ست ساعات, Brimonidine كل ٦ ساعات, كارباشول عند اللزوم, Latanoprost مرة يومياً",
        "Brimonidine Before breakfast",
        "Carbachol عند اللزوم, Tobramycin عند اللزوم, Pilocarpine كل ٦ ساعات, لاتانوبروست قبل النوم",
        "Fluorometholone As needed, Latanoprost As needed",
        "Brimonidine قبل النوم, Dorzolamide بعد الغداء",
        "دورزولاميد قبل النوم, تيمولول عند اللزوم, قطرات بريدنيزولون كل ٨ ساعات, أوفلوكساسين كل ٦ ساعات",
        "لاتانوبروست قبل النوم, توبراميسين مرة يومياً, كارباشول كل ٦ ساعات"
    ]

    # Initialize the NER system
    ner_system = MedicalPrescriptionNER()

    # Option 1: Use regex-based extraction (faster for prototyping)
    print("Processing prescriptions using regex-based extraction...")
    results = ner_system.batch_process(prescriptions)

    # Display results
    for i, result in enumerate(results):
        print(f"\nPrescription {i + 1}:")
        print(f"Text: {result['text']}")
        print("Medication Schedule:")
        for pair in result["medication_schedule"]:
            print(f"  - Medication: {pair['medication']}, Frequency: {pair['frequency']} (Confidence: {pair['confidence']:.2f})")

    # Option 2: Load a pre-trained model (if available)
    # Uncomment the following lines to use a pre-trained model instead of regex:
    # print("\nLoading pre-trained model...")
    # ner_system.load_pretrained_model()
    # results = ner_system.batch_process(prescriptions)

    # Option 3: Train a custom model (if labeled data is available)
    # Uncomment the following lines to prepare training data and train a model:
    # print("\nPreparing training data...")
    # training_data = ner_system.prepare_training_data(prescriptions)
    # print("\nTraining spaCy model...")
    # ner_system.train_spacy_model(training_data)
    # print("\nTraining transformer model...")
    # ner_system.train_transformer_model(training_data)

if __name__ == "__main__":
    main()

Processing prescriptions using regex-based extraction...

Prescription 1:
Text: Carbachol Every 8 hours
Medication Schedule:
  - Medication: Carbachol, Frequency: Every 8 hours (Confidence: 0.99)

Prescription 2:
Text: Ofloxacin كل ست ساعات, Brimonidine كل ٦ ساعات, كارباشول عند اللزوم, Latanoprost مرة يومياً
Medication Schedule:
  - Medication: Ofloxacin, Frequency: every 6 hours (Confidence: 0.99)
  - Medication: Brimonidine, Frequency: every 6 hours (Confidence: 0.99)
  - Medication: Carbachol, Frequency: as needed (Confidence: 0.99)
  - Medication: Latanoprost, Frequency: once daily (Confidence: 0.99)

Prescription 3:
Text: Brimonidine Before breakfast
Medication Schedule:
  - Medication: Brimonidine, Frequency: Before breakfast (Confidence: 0.99)

Prescription 4:
Text: Carbachol عند اللزوم, Tobramycin عند اللزوم, Pilocarpine كل ٦ ساعات, لاتانوبروست قبل النوم
Medication Schedule:
  - Medication: Carbachol, Frequency: as needed (Confidence: 0.99)
  - Medication: Tobramycin, Frequency

# try with NER model

In [ ]:
import json
import re
import spacy
import random
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments

# Define the NER system class
class MedicalPrescriptionNER:
    def __init__(self):
        self.medication_list = ["Carbachol", "Ofloxacin", "Brimonidine", "Latanoprost", "Pilocarpine", "Tobramycin", "Fluorometholone", "Dorzolamide", "Timolol", "Prednisolone"]
        self.frequency_patterns = [
            (r'كل\s*(\d+)\s*ساعات', 'every \1 hours'),
            (r'قبل النوم', 'before bed'),
            (r'بعد الغداء', 'after lunch'),
            (r'مرة يومياً', 'once daily'),
            (r'عند اللزوم', 'as needed'),
            (r'Every (\d+) hours', 'every \1 hours'),
            (r'Before breakfast', 'before breakfast'),
            (r'As needed', 'as needed')
        ]
    
    def extract_entities(self, text):
        entities = []
        for med in self.medication_list:
            if med in text:
                entities.append((med, 'MEDICATION'))
        for pattern, freq in self.frequency_patterns:
            matches = re.findall(pattern, text)
            if matches:
                for match in matches:
                    extracted_freq = freq.replace('\\1', match) if '\\1' in freq else freq
                    entities.append((extracted_freq, 'FREQUENCY'))
        return entities
    
    def batch_process(self, prescriptions):
        processed_data = []
        for text in prescriptions:
            entities = self.extract_entities(text)
            processed_data.append({"text": text, "entities": entities})
        return processed_data

def prepare_training_data(processed_data):
    labeled_data = []
    for entry in processed_data:
        text = entry['text']
        labels = ['O'] * len(text)
        for entity, label in entry['entities']:
            start_idx = text.find(entity)
            if start_idx != -1:
                for i in range(start_idx, start_idx + len(entity)):
                    labels[i] = 'B-' + label if i == start_idx else 'I-' + label
        labeled_data.append({"tokens": list(text), "labels": labels})
    return labeled_data

def train_ner_model(training_data):
    model_name = "distilbert-base-multilingual-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)
    
    tokenized_inputs = tokenizer([d['tokens'] for d in training_data], truncation=True, padding=True, is_split_into_words=True)
    labels = [[0 if l == 'O' else 1 for l in d['labels']] for d in training_data]
    
    dataset = [{"input_ids": tokenized_inputs['input_ids'][i], "attention_mask": tokenized_inputs['attention_mask'][i], "labels": labels[i]} for i in range(len(labels))]
    
    training_args = TrainingArguments(output_dir="./ner_model", num_train_epochs=3, per_device_train_batch_size=4)
    trainer = Trainer(model=model, args=training_args, train_dataset=dataset)
    trainer.train()
    model.save_pretrained("./ner_model")

def main():
    prescriptions = [
        "Carbachol Every 8 hours",
        "Ofloxacin كل ست ساعات, Brimonidine كل ٦ ساعات, كارباشول عند اللزوم, Latanoprost مرة يومياً",
        "Brimonidine Before breakfast",
        "Carbachol عند اللزوم, Tobramycin عند اللزوم, Pilocarpine كل ٦ ساعات, لاتانوبروست قبل النوم",
        "Fluorometholone As needed, Latanoprost As needed",
        "Brimonidine قبل النوم, Dorzolamide بعد الغداء",
        "دورزولاميد قبل النوم, تيمولول عند اللزوم, قطرات بريدنيزولون كل ٨ ساعات, أوفلوكساسين كل ٦ ساعات",
        "لاتانوبروست قبل النوم, توبراميسين مرة يومياً, كارباشول كل ٦ ساعات"
    ]

    ner_system = MedicalPrescriptionNER()
    results = ner_system.batch_process(prescriptions)
    training_data = prepare_training_data(results)
    
    with open("training_data.json", "w", encoding='utf-8') as f:
        json.dump(training_data, f, ensure_ascii=False, indent=4)
    
    print("Training data saved! Starting model training...")
    train_ner_model(training_data)
    print("Model training completed!")

if __name__ == "__main__":
    main()


In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Load the trained model and tokenizer
model_name = "med_ner_model"  # Change this to your actual model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Initialize a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)


In [ ]:
def predict_medications(text):
    ner_results = ner_pipeline(text)
    
    extracted_entities = []
    current_entity = {"word": "", "entity": ""}
    
    for entity in ner_results:
        word = entity["word"].replace("▁", "")  # Handling subword tokenization artifacts
        if entity["entity"].startswith("B-"):  # Beginning of a new entity
            if current_entity["word"]:  # Save the previous entity
                extracted_entities.append(current_entity)
            current_entity = {"word": word, "entity": entity["entity"][2:]}
        elif entity["entity"].startswith("I-") and current_entity["entity"]:  # Inside the same entity
            current_entity["word"] += " " + word
    
    # Add the last entity
    if current_entity["word"]:
        extracted_entities.append(current_entity)

    return extracted_entities

# Example prediction
prescription_text = "Brimonidine كل ٦ ساعات, Carbachol عند اللزوم, Latanoprost مرة يومياً"
predictions = predict_medications(prescription_text)

# Display results
for entity in predictions:
    print(f"Extracted: {entity['word']} - Label: {entity['entity']}")